In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import torch
import torch.nn as nn

import torch.nn.functional as F
import dgl
import dgl.nn as dglnn
import torch.nn as nn
from dgl import AddSelfLoop

from model_2d.NMR_gcn import NMR_GCN

from train_evaluate_2d import NMR_prediction

from preprocess.build_adjaency_matrix.create_adjaency_matrix_glycoscience import build_adjacency_matrix

from preprocess.create_graph.create_graph_data_glycosciencedb import create_graph

# from dgl import save_graphs, load_graphs
# from tqdm import tqdm
from dgl import save_graphs, load_graphs


In [2]:
g = load_graphs('glycosciencedb/graph_data/data.bin')[0][0]

In [3]:
num_epoch = 1000
lr = 1e-3

atom_dim = 256
mono_dim = 128
ab_dim = 64
dl_dim = 64
pf_dim = 64

In [4]:
in_size = atom_dim + mono_dim + ab_dim + dl_dim + pf_dim
# in_size = atom_dim

hidden_size_1 = int(in_size / 2)
hidden_size_2 = 256
hidden_size_3 = 128
hidden_size_4 = 32

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

g = g.to(device)
features = g.ndata["feat"]
labels = g.ndata["shift_value"]
# masks = g.ndata['train_mask'], g.ndata['test_mask']


# masks = g.ndata['train_hydrogen_mask'], g.ndata['test_hydrogen_mask']
masks = g.ndata['train_carbon_mask'], g.ndata['test_carbon_mask']
# masks = g.ndata['train_mask'], g.ndata['test_mask']

print(features.dtype)
print(labels.dtype)
# model = NMR_GCN(in_size=576, hid_size=[256, 128, 64, 32], out_size=1).to(device)
model = NMR_GCN(in_size=in_size, hid_size=[hidden_size_1, hidden_size_2,
                                       hidden_size_3, hidden_size_4], out_size=1).to(device)

torch.float32
torch.float32


In [6]:
model.load_state_dict(torch.load('transfer_learning_results/2D_godess/Model_Godess_carbon.pt'))

<All keys matched successfully>

In [7]:
model.parameters

<bound method Module.parameters of NMR_GCN(
  (layers): ModuleList(
    (0): GraphConv(in=288, out=256, normalization=both, activation=<function relu at 0x7f587845df80>)
    (1): GraphConv(in=256, out=128, normalization=both, activation=None)
  )
  (front_dense1): Linear(in_features=576, out_features=288, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (back_dense1): Linear(in_features=128, out_features=32, bias=True)
  (back_dense2): Linear(in_features=32, out_features=1, bias=True)
)>

In [8]:
for name, para in model.named_parameters():
    print("-"*20)
    print(f"name: {name}")
    if name not in ['back_dense2.weight', 'back_dense2.bias', 'back_dense1.weight', 'back_dense1.bias', 
                   'front_dense1.weight', 'front_dense1.bias']:
        para.requires_grad = False

--------------------
name: layers.0.weight
--------------------
name: layers.0.bias
--------------------
name: layers.1.weight
--------------------
name: layers.1.bias
--------------------
name: front_dense1.weight
--------------------
name: front_dense1.bias
--------------------
name: back_dense1.weight
--------------------
name: back_dense1.bias
--------------------
name: back_dense2.weight
--------------------
name: back_dense2.bias


In [9]:
NMR_prediction = NMR_prediction(results_dir='transfer_learning_results/2D_glycoscience_finetuned/training_carbon.csv',
                                results_dir_test = 'transfer_learning_results/2D_glycoscience_finetuned/testing_carbon.csv',
                                model_dir='transfer_learning_results/2D_glycoscience_finetuned/Model_Godess_carbon.pt', 
                               num_epoch = num_epoch, 
                               lr = lr)


print("Training...")
NMR_prediction.train(g, features, labels, masks, model)

# test the model
print("Testing...")
saved_model = NMR_GCN(in_size=in_size, hid_size=[hidden_size_1, hidden_size_2,
                                                 hidden_size_3, hidden_size_4], out_size=1).to(device)
saved_model.load_state_dict(torch.load(NMR_prediction.model_dir))

# acc = NMR_prediction.evaluate(g, features, labels, masks[0], saved_model, print_out=True)
acc1 = NMR_prediction.evaluate(g, features, labels, masks[0], saved_model, save_train=True, save_test = False)

acc2 = NMR_prediction.evaluate(g, features, labels, masks[1], saved_model, save_train=False, save_test = True)


print("train RMSE {:.4f}".format(acc1))
print("test RMSE {:.4f}".format(acc2))

Training...


  1%|▍                                        | 10/1000 [00:00<01:01, 15.97it/s]

5279
1231
Epoch 00000 | Loss 2050.2131 | train_RMSE 31.2672 | test_RMSE 32.0057 
5279
1231
Epoch 00001 | Loss 999.1255 | train_RMSE 25.6496 | test_RMSE 25.8010 
5279
1231
Epoch 00002 | Loss 699.0444 | train_RMSE 24.3193 | test_RMSE 24.3182 
5279
1231
Epoch 00003 | Loss 638.7465 | train_RMSE 22.8686 | test_RMSE 22.3124 
5279
1231
Epoch 00004 | Loss 554.3250 | train_RMSE 21.6616 | test_RMSE 20.2474 
5279
1231
Epoch 00005 | Loss 505.6165 | train_RMSE 20.9268 | test_RMSE 19.0221 
5279
1231
Epoch 00006 | Loss 480.6775 | train_RMSE 19.7658 | test_RMSE 18.0640 
5279
1231
Epoch 00007 | Loss 420.9486 | train_RMSE 18.0969 | test_RMSE 16.9389 
5279
1231
Epoch 00008 | Loss 359.9731 | train_RMSE 16.4621 | test_RMSE 15.7978 
5279
1231
Epoch 00009 | Loss 299.3430 | train_RMSE 15.3284 | test_RMSE 14.8613 
5279
1231
Epoch 00010 | Loss 259.2600 | train_RMSE 14.5562 | test_RMSE 14.0564 
5279
1231
Epoch 00011 | Loss 239.9210 | train_RMSE 13.9095 | test_RMSE 13.2924 
5279
1231
Epoch 00012 | Loss 217.4686 |

  3%|█▏                                       | 28/1000 [00:01<00:23, 41.59it/s]

5279
1231
Epoch 00017 | Loss 186.4678 | train_RMSE 12.0313 | test_RMSE 11.6560 
5279
1231
Epoch 00018 | Loss 168.0680 | train_RMSE 11.7273 | test_RMSE 11.4010 
5279
1231
Epoch 00019 | Loss 160.0071 | train_RMSE 11.4550 | test_RMSE 11.1827 
5279
1231
Epoch 00020 | Loss 160.8981 | train_RMSE 11.2380 | test_RMSE 11.0276 
5279
1231
Epoch 00021 | Loss 152.8320 | train_RMSE 11.0499 | test_RMSE 10.9147 
5279
1231
Epoch 00022 | Loss 147.6722 | train_RMSE 10.8255 | test_RMSE 10.7852 
5279
1231
Epoch 00023 | Loss 148.2914 | train_RMSE 10.5519 | test_RMSE 10.6168 
5279
1231
Epoch 00024 | Loss 132.0325 | train_RMSE 10.2704 | test_RMSE 10.4338 
5279
1231
Epoch 00025 | Loss 134.3258 | train_RMSE 10.0279 | test_RMSE 10.2685 
5279
1231
Epoch 00026 | Loss 125.5877 | train_RMSE 9.8420 | test_RMSE 10.1286 
5279
1231
Epoch 00027 | Loss 124.8492 | train_RMSE 9.6918 | test_RMSE 9.9974 
5279
1231
Epoch 00028 | Loss 128.0743 | train_RMSE 9.5426 | test_RMSE 9.8448 
5279
1231
Epoch 00029 | Loss 115.1513 | train

...

5279
1231
Epoch 00971 | Loss 21.7538 | train_RMSE 3.2123 | test_RMSE 4.3017 
5279
1231
Epoch 00972 | Loss 23.7563 | train_RMSE 3.3272 | test_RMSE 4.3547 
5279
1231
Epoch 00973 | Loss 24.5760 | train_RMSE 3.2397 | test_RMSE 4.3123 
5279
1231
Epoch 00974 | Loss 21.3395 | train_RMSE 3.2277 | test_RMSE 4.3364 
5279
1231
Epoch 00975 | Loss 22.1681 | train_RMSE 3.3272 | test_RMSE 4.4424 
5279
1231
Epoch 00976 | Loss 25.9181 | train_RMSE 3.2273 | test_RMSE 4.3564 
5279
1231
Epoch 00977 | Loss 22.5197 | train_RMSE 3.3376 | test_RMSE 4.4175 
5279
1231
Epoch 00978 | Loss 23.5572 | train_RMSE 3.2732 | test_RMSE 4.3879 
5279
1231
Epoch 00979 | Loss 23.1252 | train_RMSE 3.2261 | test_RMSE 4.3894 
5279
1231
Epoch 00980 | Loss 23.0141 | train_RMSE 3.2476 | test_RMSE 4.4202 
5279
1231
Epoch 00981 | Loss 22.9120 | train_RMSE 3.2053 | test_RMSE 4.3607 
5279
1231
Epoch 00982 | Loss 23.0839 | train_RMSE 3.2405 | test_RMSE 4.3559 
5279
1231
Epoch 00983 | Loss 23.9954 | train_RMSE 3.2794 | test_RMSE 4.3610 

 99%|███████████████████████████████████████▋| 992/1000 [00:14<00:00, 65.99it/s]

1231
Epoch 00984 | Loss 24.0281 | train_RMSE 3.2440 | test_RMSE 4.3269 
5279
1231
Epoch 00985 | Loss 22.0200 | train_RMSE 3.2329 | test_RMSE 4.3125 
5279
1231
Epoch 00986 | Loss 21.2371 | train_RMSE 3.2460 | test_RMSE 4.3190 
5279
1231
Epoch 00987 | Loss 23.0856 | train_RMSE 3.2391 | test_RMSE 4.3068 
5279
1231
Epoch 00988 | Loss 24.7215 | train_RMSE 3.2571 | test_RMSE 4.3033 
5279
1231
Epoch 00989 | Loss 23.6180 | train_RMSE 3.2452 | test_RMSE 4.2927 
5279
1231
Epoch 00990 | Loss 21.8416 | train_RMSE 3.2264 | test_RMSE 4.2816 
5279
1231
Epoch 00991 | Loss 22.0781 | train_RMSE 3.2097 | test_RMSE 4.2733 
5279
1231
Epoch 00992 | Loss 23.4988 | train_RMSE 3.2028 | test_RMSE 4.2793 
5279
1231
Epoch 00993 | Loss 22.1148 | train_RMSE 3.2086 | test_RMSE 4.2921 
5279
1231
Epoch 00994 | Loss 24.7896 | train_RMSE 3.2600 | test_RMSE 4.3211 
5279
1231
Epoch 00995 | Loss 25.0262 | train_RMSE 3.2451 | test_RMSE 4.3264 
5279
1231
Epoch 00996 | Loss 22.6883 | train_RMSE 3.2324 | test_RMSE 4.3612 
5279

100%|███████████████████████████████████████| 1000/1000 [00:14<00:00, 68.18it/s]


5279
1231
Epoch 00998 | Loss 22.9159 | train_RMSE 3.2018 | test_RMSE 4.3282 
5279
1231
Epoch 00999 | Loss 22.5085 | train_RMSE 3.2172 | test_RMSE 4.3227 
best loss: 100000
Testing...
1231
5279
train RMSE 3.2018
test RMSE 4.3282


In [10]:
# 1000 epoch
with_fine_tunned = np.array([4.3208, 4.3299, 4.2939, 4.3341, 4.3345])
np.mean(with_fine_tunned), np.std(with_fine_tunned)

(4.32264, 0.015192577134903913)